In [1]:
from data_preprocessing import *

30680
rose
{1: 'B-art', 2: 'B-geo', 3: 'B-gpe', 4: 'B-nat', 5: 'I-geo', 6: 'B-per', 7: 'I-gpe', 8: 'I-org', 9: 'I-art', 10: 'I-tim', 11: 'O', 12: 'B-org', 13: 'I-eve', 14: 'B-eve', 15: 'B-tim', 16: 'I-per', 17: 'I-nat', 0: '--PADDING--'}
[('Thousands', 'NNS', 'O'), ('of', 'IN', 'O'), ('demonstrators', 'NNS', 'O'), ('have', 'VBP', 'O'), ('marched', 'VBN', 'O'), ('through', 'IN', 'O'), ('London', 'NNP', 'B-geo'), ('to', 'TO', 'O'), ('protest', 'VB', 'O'), ('the', 'DT', 'O'), ('war', 'NN', 'O'), ('in', 'IN', 'O'), ('Iraq', 'NNP', 'B-geo'), ('and', 'CC', 'O'), ('demand', 'VB', 'O'), ('the', 'DT', 'O'), ('withdrawal', 'NN', 'O'), ('of', 'IN', 'O'), ('British', 'JJ', 'B-gpe'), ('troops', 'NNS', 'O'), ('from', 'IN', 'O'), ('that', 'DT', 'O'), ('country', 'NN', 'O'), ('.', '.', 'O')]


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data

In [3]:
Xtrain = torch.tensor(X_train[:50],dtype=torch.long)
Xtest = torch.tensor(X_test[:50],dtype=torch.long)
ytrain = torch.tensor(y_train[:50],dtype=torch.long)
ytest = torch.tensor(y_test[:50], dtype=torch.long)
Xdev = torch.tensor(X_dev[:50],dtype=torch.long)
ydev = torch.tensor(y_dev[:50],dtype=torch.long)

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [5]:
# the pre-processing steps for data before beginning training the model
loader_train = data.DataLoader(data.TensorDataset(Xtrain, ytrain), shuffle=True, batch_size=4)
loader_dev = data.DataLoader(data.TensorDataset(Xdev, ydev), shuffle=True, batch_size=4)
loader_test = data.DataLoader(data.TensorDataset(Xtest, ytest), shuffle=True, batch_size=4)


In [6]:
class BiLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=2, bidirectional=True, dropout=0.5)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        
        
    def forward(self, text):
        embedded = self.embedding(text)
        output,_ = self.lstm(embedded)
        
       
        tag_scores = self.fc(output)
        return tag_scores

In [7]:
EMBEDDING_DIM = 100
HIDDEN_DIM = 128

OUTPUT_DIM = TAG_COUNT
#NUM_LAYERS=2

In [8]:
"""class EarlyStopping:
    def __init__(self, tolerance=2, min_delta=0):

        self.tolerance = tolerance
        self.min_delta = min_delta
        self.counter = 0
        self.early_stop = False

    def __call__(self, train_loss, validation_loss):
        if (validation_loss - train_loss) > self.min_delta:
            self.counter +=1
            if self.counter >= self.tolerance:  
                self.early_stop = True """

'class EarlyStopping:\n    def __init__(self, tolerance=2, min_delta=0):\n\n        self.tolerance = tolerance\n        self.min_delta = min_delta\n        self.counter = 0\n        self.early_stop = False\n\n    def __call__(self, train_loss, validation_loss):\n        if (validation_loss - train_loss) > self.min_delta:\n            self.counter +=1\n            if self.counter >= self.tolerance:  \n                self.early_stop = True '

In [9]:
model_lstm=BiLSTM(len_uniq_words,EMBEDDING_DIM,HIDDEN_DIM,OUTPUT_DIM).to(device)

criterion = nn.CrossEntropyLoss()
optimizer_lstm = optim.Adam(model_lstm.parameters())

In [10]:
num_epochs=10
for epoch in range(num_epochs):
    model_lstm.train()
    train_loss=0.0
    #print(epoch)
    for x_batch, y_batch in loader_train:
        
        optimizer_lstm.zero_grad()
        tag_scores = model_lstm(x_batch)
        predictions=tag_scores.view(-1,tag_scores.shape[-1])
        
        tags=y_batch.view(-1)
        #print("true tag",tags)
        loss = criterion(predictions, tags)
        loss.backward()
        optimizer_lstm.step()
        train_loss += loss.item()
    train_loss/=len(loader_train)
    

    #Evaluation phase
    model_lstm.eval()
    dev_loss=0.0
    with torch.no_grad():
      for x_batch, y_batch in loader_dev:
        tag_scores = model(x_batch)
        predictions=tag_scores.view(-1,tag_scores.shape[-1])
        
        tags=y_batch.view(-1)
        loss = criterion(predictions, tags)

        dev_loss+=loss.item()
    dev_loss /= len(loader_dev)
    
        
   
    #early_stopping = EarlyStopping(tolerance=2, min_delta=0)


    #early_stopping(train_loss, dev_loss)
  
    print(f"Epoch {epoch + 1}, Train Loss: {train_loss}, Dev Loss: {dev_loss}")
    #if early_stopping.early_stop:
        #print("We are at epoch:", epoch)
        #break
    

NameError: name 'model' is not defined

In [11]:
def decode_tag(predictions,idtotag):
  decoded_tags=[idtotag[int(p)] for p in predictions]
  return decoded_tags

In [12]:
# Evaluate the model on test data
model_lstm.eval()
all_preds=[]
all_true_tags=[]

with torch.no_grad():
  for x_batch, y_batch in loader_test:
    tag_scores = model_lstm(x_batch)
    predictions=tag_scores.view(-1,tag_scores.shape[-1])
    max_pred=predictions.argmax(dim=-1)
    true_tags=y_batch.view(-1)
    decoded_pred=decode_tag(max_pred,id2tag)
    decoded_true_tag=decode_tag(true_tags,id2tag)
    all_preds.extend(decoded_pred)
    all_true_tags.extend(decoded_true_tag)
#print(all_preds)
#print(all_true_tags)
accuracy= accuracy_score(all_true_tags,all_preds)
#precision = precision_score(all_true_tags, all_preds, average="weighted")
#recall=recall_score(all_true_tags, all_preds)
#f1=f1_score(all_true_tags, all_preds)

print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.82
